In [ ]:
!pip install pandas 
!pip install numpy
!pip install pyarrow 
!pip install matplotlib

In [ ]:
import pandas as pd
import re
import numpy as np
import json
import os
import shutil
import matplotlib.pyplot as plt

In [ ]:
#Bereinigung der Datenbank
# Lade die Parquet-Datei von dem angegebenen Pfad
file_path = "   " # Pfad zur Parquet-Datei

# Lade Parquet-Datei
df = pd.read_parquet(file_path)

# Extrahiere die relevanten Spalten
df_filtered = df[['merge_id', 'Jahr', 'Verfasser']]

# Funktion zur Bereinigung der Jahr-Spalte
def clean_year(year):
    if pd.isna(year):
        return np.nan
    cleaned = re.sub(r'[\[\]()?;]', '', str(year)) # Entferne Sonderzeichen
    cleaned = re.sub(r'\b(ca\.|um|circa|vor|nach|erschienen|nicht ermittelbar)\b', '', cleaned, flags=re.IGNORECASE) # Entferne überflüssige Wörter
    cleaned = re.sub(r'\s+', '', cleaned) # Entferne überflüssige Leerzeichen
    match = re.search(r'\b\d{4}\b', cleaned)
    return match.group(0) if match else np.nan

# Bereinigen der 'Jahr'-Spalte und speichern im DataFrame
df_filtered['Jahr'] = df_filtered['Jahr'].apply(clean_year)

In [ ]:
# Optional: Zeige die bereinigten eindeutigen Jahre
unique_years_sorted = sorted(df_filtered['Jahr'].dropna().unique(), key=lambda x: int(x))
print("Bereinigte eindeutige Jahre in der Datenbank:")
for year in unique_years_sorted:
    print(year)

In [ ]:
# Speichere die bereinigte Datenbank an einem neuen Ort
output_path = "   " # Zielpfad für die bereinigte Parquet-Datei
df_filtered.to_parquet(output_path, index=False)
print(f"Bereinigte Datenbank erfolgreich gespeichert unter: {output_path}")

In [ ]:
# Filterung: Bücher zwischen 1800 und 1900
filtered_df = df_filtered[(df_filtered['Jahr'].astype(str) >= '1800') & (df_filtered['Jahr'].astype(str) <= '1900')]

# Extrahiere nur die Buchnummern und das Jahr
filtered_data = filtered_df[['merge_id', 'Jahr']].to_dict(orient='records')

# Speichern als JSON
output_json_path = "   " # Pfad zur Ausgabe-JSON-Datei
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)
print("JSON-Datei wurde erfolgreich gespeichert:", output_json_path)

In [ ]:
# Kopiere Bücherordner
json_path = "   "  # Pfad zur JSON-Datei mit Buchnummern
source_base_path = "   " # Quellpfad für die Bücherordner
destination_path = "   " # Zielpfad für die kopierten Ordner

os.makedirs(destination_path, exist_ok=True)

# Lade JSON-Datei
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
book_ids = {str(entry['merge_id']) for entry in data}

# Durchlaufe alle Ordner im Dataset und kopiere relevante
for folder_name in os.listdir(source_base_path):
    folder_path = os.path.join(source_base_path, folder_name)
    if folder_name in book_ids and os.path.isdir(folder_path):
        dest_folder_path = os.path.join(destination_path, folder_name)
        shutil.copytree(folder_path, dest_folder_path)
        print(f"Ordner '{folder_name}' wurde erfolgreich kopiert.")

print("Alle passenden Ordner wurden erfolgreich kopiert.")

In [ ]:
# Erstelle ein Diagramm für den Tier-Faktor
csv_path = "   " # Pfad zur CSV-Datei mit Tier-Faktor-Daten
df = pd.read_csv(csv_path)

plt.figure(figsize=(10, 6))
plt.plot(df['Erscheinungsjahr'], df['Durchschnittlicher Tier-Faktor'], marker='o', linestyle='-')
plt.xlabel('Jahr')
plt.ylabel('Durchschnittlicher Tier-Faktor')
plt.title('Durchschnittlicher Tier-Faktor pro Jahr (1800-1900)')
plt.grid(True)
plt.show()

In [ ]:
# Ausgabe der Autoren von 1812
database_path = "   "  # Pfad zur Datenbank-Datei
df = pd.read_parquet(database_path)
authors_1812 = df[df['Jahr'] == '1812']['Verfasser'].unique()
print("Autoren, die 1812 veröffentlicht haben:")
for author in authors_1812:
    print(author)

In [ ]:
# Berechnung der Anzahl der Bücher pro Jahr
books_per_year = df.groupby('Jahr')['merge_id'].nunique()
plt.figure(figsize=(12, 6))
plt.plot(books_per_year.index, books_per_year.values, marker='o', linestyle='-')
plt.xlabel('Jahr')
plt.ylabel('Anzahl der Bücher')
plt.title('Anzahl der veröffentlichten Bücher pro Jahr')
plt.grid(True)
plt.show()

In [ ]:
# Bereinigen von Namen in der Datenbankdatei
def sanitize_name(name):
    if isinstance(name, str):
        return re.sub(r'[<>:"/\\|?*]', '_', name)
    return name

cleaned_file_path = "   " # Pfad zur bereinigten Datei
data = pd.read_parquet(database_path)

if 'Verfasser' in data.columns:
    data['Verfasser'] = data['Verfasser'].apply(sanitize_name)
if 'merge_id' in data.columns:
    data['merge_id'] = data['merge_id'].apply(sanitize_name)

data.to_parquet(cleaned_file_path, index=False)
print(f"Die bereinigte .parquet-Datei wurde gespeichert unter: {cleaned_file_path}")